In [1]:
import sklearn 

from sklearn import datasets                
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold               
from sklearn.metrics import accuracy_score                 
from sklearn.metrics import confusion_matrix               

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#read file
datasetpath = 'pulmonary_adenocarcinoma_dataset.csv'
dataset = pd.read_csv(datasetpath)

X = np.vstack((dataset['lobular_speciticular'], dataset['air_bronchus_sign'], dataset['vessel_passthrough'], dataset['long_diameter'], dataset['avarage_CT_value'])).T

y = np.array(dataset['pathology'])


def simplepredict(model):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)   #, random_state=46
    y_pred = model.predict(X_train)
    print(accuracy(y_pred,y_test))



def Kfoldpredict(model):
    kf = KFold(5,shuffle=True)
    fold = 1
    overallaccuracy = 0

    for train_index, validate_index in kf.split(X,y):
        model.fit(X[train_index],y[train_index])
        y_test = y[validate_index]
        y_pred = model.predict(X[validate_index])
        #print(y_test)
        #print(y_pred)
        #print(f"Fold #{fold}, Training Size: {len(trainDF)}, Validation Size: {len(validateDF)}")
        print(f"Fold #{fold}, Training Size: {len(X[train_index])}, Validation Size: {len(X[validate_index])}")
        print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
        fold += 1
        overallaccuracy = overallaccuracy  + accuracy_score(y_test, y_pred)
    print(overallaccuracy/5)

In [ ]:


from sklearn.linear_model import LogisticRegression, LinearRegression

logisticregressionmodel = LogisticRegression()
Kfoldpredict(logisticregressionmodel)

'''
logisticregressionmodel.fit(X_train, y_train)
y_pred = logisticregressionmodel.predict(X_test)
accuracy_score(y_test, y_pred)
'''

In [ ]:


from sklearn.svm import SVC  

svcmodel = SVC(kernel='linear', C=1E10)
svcmodel.fit(X_train, y_train)
y_pred = svcmodel.predict(X_train)
Kfoldpredict(svcmodel)

In [ ]:


from sklearn.tree import DecisionTreeClassifier

decisiontreemodel = DecisionTreeClassifier(criterion = 'entropy')
decisiontreemodel.fit(X_train,y_train)
y_pred = decisiontreemodel.predict(X_test)

Kfoldpredict(decisiontreemodel)

In [ ]:


from sklearn.ensemble import RandomForestClassifier

randomforestmodel = RandomForestClassifier(n_estimators=10,criterion="entropy")
randomforestmodel.fit(X_train, y_train)
y_pred = randomforestmodel.predict(X_test)

Kfoldpredict(randomforestmodel)

In [ ]:


from sklearn.neighbors import KNeighborsClassifier

knnmodel = KNeighborsClassifier()
knnmodel.fit(X_train, y_train)
y_pred = knnmodel.predict(X_test)

Kfoldpredict(knnmodel)

In [ ]:


from sklearn.naive_bayes import GaussianNB

bayesianmodel = GaussianNB()
bayesianmodel.fit(X, y)
y_pred = bayesianmodel.predict(X_test)

Kfoldpredict(bayesianmodel)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense#, Activation

import numpy as np

def onehot(X):                                 #Onehot
    T = np.zeros((X.size, 4))
    for idx, row in enumerate(T):
        row[X[idx]-1] = 1
    return T

y = onehot(y)

model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

kf = KFold(5)

overallaccuracy = 0

for train, test in kf.split(X):
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

    model.fit(X_train,y_train,verbose=2,epochs=1000)
    pred = model.predict(X_test)
    pred = np.argmax(pred,axis=1)
    y_compare = np.argmax(y_test,axis=1) 
    score = metrics.accuracy_score(y_compare, pred)
    overallaccuracy = overallaccuracy + score
    print("Accuracy score: {}".format(score))
print ('overall accuracy:', overallaccuracy/5)
